# frames -> fix -> object -> increase -> video

# 임포트

In [1]:
import cv2
import time
import os
import numpy as np
import pandas as pd
from PIL import Image
import PIL.ImageOps  
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')

# 비디오 읽어오기

In [2]:
cap = cv2.VideoCapture("10.mp4")

length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

print("length : ", length)
print("height : ", height)
print("width : ", width)

length :  0
height :  0
width :  0


# 파일 안의 데이터 삭제

In [3]:
def removeAllFile(filePath):
    if os.path.exists(filePath):
        for file in os.scandir(filePath):
            os.remove(file.path)
        return "Remove All File"
    else:
        return "Directory Not Found"

# 비디오 프레임으로 쪼개기

In [4]:
def video_to_frames(video, path_output_dir):
    vidcap = cv2.VideoCapture(video)
    count = 0
    while vidcap.isOpened():
        success, image = vidcap.read()
        if success:
            cv2.imwrite(os.path.join(path_output_dir, '%d.png') % count, image)
            count += 1
        else:
            break
    cv2.destroyAllWindows()
    vidcap.release()

In [5]:
removeAllFile(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\frames')

'Directory Not Found'

In [6]:
video_to_frames(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\10.mp4',
                r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\frames')

# 사진 자르기

In [7]:
removeAllFile(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\fix')

'Directory Not Found'

In [8]:
count = 0
area = (0, 0, 1280, 570)

for i in range(0, length-1):
    img = Image.open(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\frames\{}.png".format(i))
    cropped_img = img.crop(area)
    cropped_img.save(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\fix\{}.png".format(i))

## 영역 인식

In [9]:
def region_of_interest(img, vertices, color3=(255,255,255), color1=255): # ROI 셋팅

    mask = np.zeros_like(img) # mask = img와 같은 크기의 빈 이미지
    
    if len(img.shape) > 2: # Color 이미지(3채널)라면 :
        color = color3
    else: # 흑백 이미지(1채널)라면 :
        color = color1
        
    # vertices에 정한 점들로 이뤄진 다각형부분(ROI 설정부분)을 color로 채움 
    cv2.fillPoly(mask, vertices, color)
    
    # 이미지와 color로 채워진 ROI를 합침
    ROI_image = cv2.bitwise_and(img, mask)
    return ROI_image

In [10]:
def mark_img(img, blue_threshold=200, green_threshold=200, red_threshold=200): # 흰색 차선 찾기

    #  BGR 제한 값
    bgr_threshold = [blue_threshold, green_threshold, red_threshold]

    # BGR 제한 값보다 작으면 검은색으로
    thresholds = (image[:,:,0] < bgr_threshold[0]) | (image[:,:,1] < bgr_threshold[1]) | (image[:,:,2] < bgr_threshold[2])
    mark[thresholds] = [0,0,0]
    return mark

In [11]:
#fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (640, 480))
#output1 = r'C:\Users\82107\project3\opencv-car-detection-master\opencv-car-detection-master\video\output1.mp4'
#out = cv2.VideoWriter(output1, fourcc, 30, (width,height))

## areaframes의 사진 삭제

In [12]:
removeAllFile(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\areaframes')

'Directory Not Found'

In [13]:
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
#out = cv2.VideoWriter('output.mp4', fourcc, 25.0, (640, 480))
#output1 = r'C:\Users\82107\project3\opencv-car-detection-master\opencv-car-detection-master\video\fix.mp4'
count = 0
for i in range(0, length-1):
    image = cv2.imread(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\fix\{}.png'.format(i), 1)
    height, width = image.shape[:2] # 이미지 높이, 너비

    # 사다리꼴 모형의 Points
    vertices = np.array([[(50,height),(width/2-45, height/2+60), (width/2+45, height/2+60), (width-50,height)]], dtype=np.int32)
    roi_img = region_of_interest(image, vertices, (0,0,255)) # vertices에 정한 점들 기준으로 ROI 이미지 생성

    mark = np.copy(roi_img) # roi_img 복사
    mark = mark_img(roi_img) # 흰색 차선 찾기

    # 흰색 차선 검출한 부분을 원본 image에 overlap 하기
    color_thresholds = (mark[:,:,0] == 0) & (mark[:,:,1] == 0) & (mark[:,:,2] > 200)
    image[color_thresholds] = [0,0,255]

    cv2.imshow('results',image) # 이미지 출력
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    # 출력된 이미지 areaframes에 저장
    cv2.imwrite(os.path.join(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\areaframes", '%d.png')% count, image)
    count = count + 1    
#cap.release()
#out.release()
cv2.destroyAllWindows()

## video에 저장

In [14]:
removeAllFile(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\video')

'Directory Not Found'

In [15]:
for i in range(0, length-1):
    img = Image.open(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\areaframes\{}.png'.format(i))

image_folder = r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\areaframes'
video_name = r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\video\10area.mp4'

#images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
video = cv2.VideoWriter(video_name, 0, 30, (width,height))

for i in range(0, length-1):
    video.write(cv2.imread(os.path.join(image_folder, "{}.png".format(i))))

cv2.destroyAllWindows()
video.release()

# 객체 추적 + 노란색 인식

In [16]:
removeAllFile(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\object')

'Directory Not Found'

In [17]:
# def tracking():
#     while True:
#         hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#         lower_yellow = np.array([10, 100, 100])
#         upper_yellow = np.array([40, 255, 255])  
#         mask_yello = cv2.inRange(hsv, lower_yellow, upper_yellow)
#         res = cv2.bitwise_and(img, img, mask = maks_yellow)
        
#         cv2.imshow("Yellow", res)
        
#         k = cv2.waitKey(0) & 0xFF
#         if k == 27:
#             break;
            
#     cv2.destroyAllWindows()

In [18]:
car_cascade = cv2.CascadeClassifier('cars.xml')
count = 0

# hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# lower_yellow = np.array([10, 100, 100])
# upper_yellow = np.array([40, 255, 255])    
# mask_yello = cv2.inRange(hsv, lower_yellow, upper_yellow)

for i in range(0,length-1):
    img = cv2.imread(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\fix\{}.png'.format(i), 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cars = car_cascade.detectMultiScale(gray, 1.8, 2, minSize=(60,30))
    ncars = 0
    
    for (x, y, w, h) in cars:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255), 2)
        
        x1 = x + (w/2)
        x2 = 640
        
        ncars = ncars + 1
        
        dis = abs((x1-x2)/(x-w))
        
        final = []
        final2 = []
        
        if dis<0.3:
            final.append(round(i/30, 0))
            final2 = final2 + final
#             print(dis,"-",i/30)
#             print(dis,"-",i)
#             tracking()

        
    cv2.imwrite(os.path.join(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\object", '%d.png') % count, img)
    count = count + 1    

In [19]:
a = []

for i in range(0, length):
    a.append(final2.count(i))
    
def sum_list(lst, res = 0):    
    for i in lst:
        if type(i) == list:
            res += sum_list(i)
        else:
            res += i
    return res

b = sum_list(a)/length

c = []

for j in range(1, length):
    if a[j] > b:
        c.append(round(a[j]/30, 0))
        
c.sort()

ZeroDivisionError: division by zero

In [23]:
# del final_list[-1:]
final_list = list(set(c))
final_list2 = []

for i in final_list:
    final_list2.append(time.strftime("%H:%M:%S", time.gmtime((4))))
#     final_list2.append(time.strftime("%H:%M:%S", time.gmtime((i/2))))
    
final_dfs = pd.DataFrame(final_list2, columns = ["영상 시간"])
final_dfss = final_dfs.set_index("영상 시간")
final_df = final_dfss.iloc[:1]
final_df

""
영상 시간
00:00:04


In [24]:
# img = mpimg.imread(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\test\1.png", cv2.IMREAD_COLOR)

# img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# lower_yellow = np.array([0,0,0])
# upper_yellow = np.array([100, 255, 255])  

# img_mask = cv2.inRange(img_hsv, lower_yellow, upper_yellow)

# img_result = cv2.bitwise_and(img, img, mask=img_mask)

# cv2.imwrite(os.path.join(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\test", '2.png'), img_result)

# 이미지 늘리기

In [25]:
removeAllFile(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\increase')

'Remove All File'

In [26]:
count = 0
area = (1280, 720)
for i in range(0, length-1):
    img = Image.open(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\object\{}.png".format(i))
    resized_img = img.resize(area)
    resized_img.save(r"C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\increase\{}.png".format(i))

# 비디오 만들기

In [27]:
removeAllFile(r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\video')

'Remove All File'

In [28]:
image_folder = r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\increase'
video_name = r'C:\Users\ynm64\project3\opencv-car-detection-master\opencv-car-detection-master\video\10_fix.mp4'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
video = cv2.VideoWriter(video_name, 0, 30, (width,height))

for i in range(0, length-1):
    video.write(cv2.imread(os.path.join(image_folder, "{}.png".format(i))))

cv2.destroyAllWindows()
video.release()